In [2]:
import os
import numpy as np
import cv2
from scipy import ndimage as ndi
import imageio
from os import walk
from pyefd import elliptic_fourier_descriptors
from skimage import feature
import pandas as pd
from sklearn.preprocessing import normalize 
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split,GridSearchCV  
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.metrics import confusion_matrix
import seaborn as sns; sns.set()
import collections
import joblib
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from scipy.stats import skew
from skimage import segmentation
from skimage.filters import sobel

np.random.seed(42)


In [3]:
def plt_t(title, img, cmap=None):
    plt.imshow(img, cmap=cmap)
    plt.title(title)
    plt.show()

In [4]:
%%capture
# Pre-processing of the images is done


def is_lightened(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    _, _, v = cv2.split(hsv)
    quantiles = np.quantile(v, [0.5])
    # print(quantiles)
    return quantiles[0] > 230


def segm_1(img_rgb):
  img_ycrcb = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2YCR_CB)  
  ycrcbmin = np.array((0, 133, 77))
  ycrcbmax = np.array((255, 173, 127))
  skin_ycrcb = cv2.inRange(img_ycrcb, ycrcbmin, ycrcbmax)
  kernel = np.ones((5, 5), np.uint8)
  img_erode = cv2.erode(skin_ycrcb, kernel, iterations=2)  
  holesimg = ndi.binary_fill_holes(img_erode).astype(np.int) 
  return holesimg

def make_to_left(img):
  a = img.sum(axis=0)
  a = a / a.max()
  if(a[:int(len(a)//8)].sum() < a[int(len(a)*7//8):].sum()):
    # flip the image
    return cv2.flip(img, 1)
  return img

def segm_5(img):
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mk = img_hsv > np.array([0, 0, 230])
    mk = mk.astype(np.float32)
    mask = (mk *0.5 + 0.5)
    masked = (mask* img_hsv)
    img_partly_darken = cv2.cvtColor(masked, cv2.COLOR_HSV2RGB)
    green_mask = (img_partly_darken[:, :, 0] > img_partly_darken[:, :, 1]).astype(np.float32)
    kernel = np.ones((5, 5), np.uint8)
    img_erode = cv2.erode(green_mask, kernel, iterations=2)  
    holesimg = ndi.binary_fill_holes(img_erode).astype(np.int) 
    return holesimg



def ImageSegmentation():
    path_MEN = r"./Image-Segmentation2/MEN"
    path_WOMEN = r"./Image-Segmentation2/WOMEN"
    if not os.path.exists(path_MEN):
        os.makedirs(path_MEN)
    if not os.path.exists(path_WOMEN):
        os.makedirs(path_WOMEN)
        
    lstFiles = []  # nombre de imagenes
    path = r"./dataset"

    for (path, _, archivos) in walk(path):
        for arch in archivos:
            (nomArch, ext) = os.path.splitext(arch)
            if (ext == ".JPG"):
                lstFiles.append(nomArch + ext)
                direc = path + "/" + nomArch + ext
                name = nomArch + ext
                print(path + "/" + nomArch + ext)
                try:
                  img_rgb = cv2.imread(direc)
                  img_rgb = cv2.resize(img_rgb, (461, 260))
                except:
                  continue
                
                print(path)
                # print(archivos)
                if(is_lightened(img_rgb)):
                  holesimg = segm_5(img_rgb)
                else:
                  holesimg = segm_1(img_rgb)
                
                # holesimg = segm_3(img_rgb)
                
                holesimg = make_to_left(holesimg)
                if 'woman' in name:
                  imageio.imwrite(os.path.join(path_WOMEN, name), holesimg)
                else:
                  imageio.imwrite(os.path.join(path_MEN, name), holesimg)
                  
                
ImageSegmentation()